In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, InputLayer, Conv2D, MaxPooling2D, Flatten ,BatchNormalization
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
!pip install -q -U keras-tuner
import keras_tuner as kt

In [ ]:
# MNISTデータを読込む
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# MNISTデータを加工する
img_rows, img_cols = 28, 28
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
# リストの後ろからN個の要素を削除
x_train = x_train[:-58000]
y_train = y_train[:-58000]
x_train  = x_train.astype('float32')
x_test   = x_test.astype('float32')
x_train /= 255
x_test  /= 255
y_train  = keras.utils.to_categorical(y_train, 10)
y_test   = keras.utils.to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#モデルを定義する
def build_model(hp):
  model = Sequential()
  model.add(InputLayer(input_shape=(img_rows, img_cols, 1)))
  #畳み込み層(1)
  model.add(Conv2D(filters=hp.Choice('filters1', [6, 16, 32]), kernel_size=(3, 3), activation='relu'))
  #プーリング層(1)
  model.add(MaxPooling2D(pool_size=(2, 2)))
  #畳み込み層(2)
  model.add(Conv2D(filters=hp.Choice('filters2', [16, 32, 64]), kernel_size=(3, 3), activation='relu'))
  #プーリング層(2)
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(hp.Int('dense_units', 32, 512, step=32), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(hp.Float('dropout_rate', 0.1, 0.5, step=0.1)))
  model.add(Dense(units=120, activation=hp.Choice("activation1", ["relu", "tanh"])))
  model.add(Dense(units=84, activation=hp.Choice("activation2", ["relu", "tanh"])))
  model.add(Dense(units=10, activation = 'softmax'))
  learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
  model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
  return model

In [ ]:
#ハイパーパラメータ探索を行うTunerを作成
tuner = kt.Hyperband(
    hypermodel=build_model,
    objective='val_accuracy',
    max_epochs=60,
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
#実際に探索する
tuner.search(x_train, y_train,
             validation_data=(x_test, y_test),
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10)], epochs=60)

Trial 90 Complete [00h 01m 54s]
val_accuracy: 0.9617999792098999

Best val_accuracy So Far: 0.9743000268936157
Total elapsed time: 01h 29m 29s


In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
score = best_model.evaluate(x_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9683 - loss: 0.1424
Test loss: 0.12060487270355225
Test accuracy: 0.9743000268936157
